In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression #categóricos
from sklearn.utils import resample
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler



In [14]:
perfecta = pd.read_csv("cancion_perfecta.csv")

# Codificación, balanceo, estandarización y entrenamiento del modelo

In [16]:
# Codificar la columna "Industria"
label_encoder = LabelEncoder()
perfecta['Industria_encoded'] = label_encoder.fit_transform(perfecta['Industria'])


In [65]:
# Seleccionamos las columnas con las que queremos enrenar el modelo
perfecta = perfecta[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence','Industria_encoded']]

In [18]:
# Seleccionar las características numéricas y definimos target
features = perfecta.select_dtypes(include=[np.number]).drop(columns=['Industria_encoded'])
target = perfecta['Industria_encoded']


In [19]:
# Aplicar Random Oversampling
ros = RandomOverSampler(random_state=42)


In [20]:
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

In [22]:
from collections import Counter
# Verificar la nueva distribución de clases
print("Distribución original de clases:", Counter(y_train))
print("Distribución después de Random Oversampling:", Counter(y_train_res))


Distribución original de clases: Counter({2: 641, 1: 158, 0: 73, 6: 51, 5: 10, 3: 8, 4: 1})
Distribución después de Random Oversampling: Counter({1: 641, 2: 641, 6: 641, 0: 641, 3: 641, 5: 641, 4: 641})


In [23]:
# Normalizar las características
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [24]:
# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

In [25]:
# Entrenar un modelo RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [26]:
# Evaluar el modelo
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [64]:
# Reporte de clasificación
classification_rep = classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Reporte de clasificación:\n", classification_rep)
print("Matriz de confusión:\n", conf_matrix)

Reporte de clasificación:
                                     precision    recall  f1-score   support

           Automotriz y Transporte       0.12      0.05      0.07        21
                  Consumo y Retail       0.36      0.12      0.18        42
          Entretenimiento y Medios       0.65      0.89      0.75       151
Finanzas y Servicios Profesionales       0.00      0.00      0.00         2
                          Política       0.00      0.00      0.00         0
       Salud, Bienestar y Deportes       0.00      0.00      0.00         3
       Tecnología y Comunicaciones       0.17      0.06      0.09        17

                          accuracy                           0.60       236
                         macro avg       0.19      0.16      0.16       236
                      weighted avg       0.50      0.60      0.53       236

Matriz de confusión:
 [[  1   0  19   0   0   0   1]
 [  1   5  36   0   0   0   0]
 [  3   8 135   0   1   0   4]
 [  0   0   2   0  

# ¿En que porcentaje es adecuada una canción para cada industria?

In [28]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [29]:
# Ocultamos nuestras credenciales y las definimos
load_dotenv()

True

In [30]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')


In [31]:
# Autenticación con Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [46]:
enlace = input("Por favor, introduce el enlace de la canción en Spotify: ")

In [47]:
track_id = enlace.split("/")[-1].split("?")[0]

In [48]:
audio_features = sp.audio_features(track_id)[0]

In [49]:
audio_features

{'danceability': 0.364,
 'energy': 0.275,
 'key': 5,
 'loudness': -11.832,
 'mode': 1,
 'speechiness': 0.043,
 'acousticness': 0.993,
 'instrumentalness': 0.0284,
 'liveness': 0.293,
 'valence': 0.0394,
 'tempo': 86.096,
 'type': 'audio_features',
 'id': '0wtpkz93wATDkUExJVuXEl',
 'uri': 'spotify:track:0wtpkz93wATDkUExJVuXEl',
 'track_href': 'https://api.spotify.com/v1/tracks/0wtpkz93wATDkUExJVuXEl',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0wtpkz93wATDkUExJVuXEl',
 'duration_ms': 131933,
 'time_signature': 4}

In [50]:
audio_features_df = pd.DataFrame([audio_features])

audio_features_df


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.364,0.275,5,-11.832,1,0.043,0.993,0.0284,0.293,0.0394,86.096,audio_features,0wtpkz93wATDkUExJVuXEl,spotify:track:0wtpkz93wATDkUExJVuXEl,https://api.spotify.com/v1/tracks/0wtpkz93wATD...,https://api.spotify.com/v1/audio-analysis/0wtp...,131933,4


In [51]:
audio_features_df = audio_features_df[['acousticness', 'danceability', 'energy', 'instrumentalness', 
                          'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]

In [52]:
audio_features_df_scaled = scaler.transform(audio_features_df)

In [53]:
prediccion = model.predict(audio_features_df_scaled)

In [54]:
probabilidades = model.predict_proba(audio_features_df_scaled)

In [56]:
industria_predicha = label_encoder.inverse_transform(prediccion)[0]

In [57]:
industria_predicha

'Entretenimiento y Medios'

In [58]:
probabilidades_industrias = dict(zip(label_encoder.classes_, probabilidades[0]))

In [61]:
print("Porcentaje de adecuación de la canción para cada industria:")
for industria, probabilidad in probabilidades_industrias.items():
    print(f"La adecuación de esta canción para la industria de {industria} es de un {probabilidad:.2%}")

Porcentaje de adecuación de la canción para cada industria:
La adecuación de esta canción para la industria de Automotriz y Transporte es de un 12.00%
La adecuación de esta canción para la industria de Consumo y Retail es de un 23.93%
La adecuación de esta canción para la industria de Entretenimiento y Medios es de un 43.46%
La adecuación de esta canción para la industria de Finanzas y Servicios Profesionales es de un 0.00%
La adecuación de esta canción para la industria de Política es de un 9.00%
La adecuación de esta canción para la industria de Salud, Bienestar y Deportes es de un 1.00%
La adecuación de esta canción para la industria de Tecnología y Comunicaciones es de un 10.61%


In [62]:
industrias_ordenadas = sorted(probabilidades_industrias.items(), key=lambda x: x[1], reverse=True)

top_3_industrias = industrias_ordenadas[:3]

print("Las tres industrias más adecuadas para la canción son:")
for industria, probabilidad in top_3_industrias:
    print(f"{industria}: {probabilidad:.2%}")


Las tres industrias más adecuadas para la canción son:
Entretenimiento y Medios: 43.46%
Consumo y Retail: 23.93%
Automotriz y Transporte: 12.00%
